# Cleaning Up

This notebook demonstrates how to clean up all the resources created in the previous set of notebooks.

In [2]:
import boto3
import json

In [3]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [20]:
%store -r

## Clean up campaigns

Start by cleaning up the campaigns. You need to delete campaigns before you can delete the solution versions upon which they are based. 

The code below will list all of the campaigns in your account.

In [4]:
paginator = personalize.get_paginator('list_campaigns')
for paginate_result in paginator.paginate():
    for campaign in paginate_result["campaigns"]:
        print(campaign["campaignArn"])

arn:aws:personalize:us-east-1:059124553121:campaign/newhrnncoldstart-30944
arn:aws:personalize:us-east-1:059124553121:campaign/personalize-poc-SIMS
arn:aws:personalize:us-east-1:059124553121:campaign/personalize-poc-hrnn
arn:aws:personalize:us-east-1:059124553121:campaign/personalize-poc-rerank


Look through the list of ARNs to determine which campaigns you want to delete. Then use the code below to delete the campaign by inserting the ARN.

In [5]:
personalize.delete_campaign(
    campaignArn = "arn:aws:personalize:us-east-1:059124553121:campaign/newhrnncoldstart-30944"
)
personalize.delete_campaign(
    campaignArn = "arn:aws:personalize:us-east-1:059124553121:campaign/personalize-poc-SIMS"
)
personalize.delete_campaign(
    campaignArn = "arn:aws:personalize:us-east-1:059124553121:campaign/personalize-poc-hrnn"
)
personalize.delete_campaign(
    campaignArn = "arn:aws:personalize:us-east-1:059124553121:campaign/personalize-poc-rerank"
)

{'ResponseMetadata': {'RequestId': '95c026ee-6113-4ea3-9efc-2115af3018bc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 03 Sep 2020 14:43:50 GMT',
   'x-amzn-requestid': '95c026ee-6113-4ea3-9efc-2115af3018bc',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up event trackers

Next, clean up any event trackers. The code below will list all of the event trackers in your account.

In [9]:
paginator = personalize.get_paginator('list_event_trackers')
for paginate_result in paginator.paginate():
    for event_tracker in paginate_result["eventTrackers"]:
        print(event_tracker["eventTrackerArn"])

arn:aws:personalize:us-east-1:059124553121:event-tracker/2114f82b
arn:aws:personalize:us-east-1:059124553121:event-tracker/84b78dd8
arn:aws:personalize:us-east-1:059124553121:event-tracker/c87ed6d0
arn:aws:personalize:us-east-1:059124553121:event-tracker/d6eeda4b


Look through the list of ARNs to determine which event tracker you want to delete. Then use the code below to delete the it by inserting the ARN.

In [12]:
personalize.delete_event_tracker(
    eventTrackerArn = event_tracker_arn
)

{'ResponseMetadata': {'RequestId': '23847076-85d1-47bb-82ce-7b059b47d0b8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 03 Sep 2020 14:46:09 GMT',
   'x-amzn-requestid': '23847076-85d1-47bb-82ce-7b059b47d0b8',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up Filters

Next, clean up any filters. The code below iterate over all filters you created and delete them.

In [14]:
for filter_arn in meta_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in interaction_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )

## Clean up solutions

Next, clean up the solutions. The code below will list all of the solutions in your account.

In [15]:
paginator = personalize.get_paginator('list_solutions')
for paginate_result in paginator.paginate():
    for solution in paginate_result["solutions"]:
        print(solution["solutionArn"])

arn:aws:personalize:us-east-1:059124553121:solution/newhrnncoldstart-30944
arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-hrnnhpo
arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-rerank
arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-sims
arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-userpersonalization


Look through the list of ARNs to determine which solution you want to delete. Then use the code below to delete the it by inserting the ARN.

In [16]:
personalize.delete_solution(
    solutionArn = "arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-hrnnhpo"
)
personalize.delete_solution(
    solutionArn = "arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-rerank"
)
personalize.delete_solution(
    solutionArn = "arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-sims"
)
personalize.delete_solution(
    solutionArn = "arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-userpersonalization"
)

{'ResponseMetadata': {'RequestId': '4e0d3c59-a60b-4a6d-9364-b1fc171b00cd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 03 Sep 2020 15:05:35 GMT',
   'x-amzn-requestid': '4e0d3c59-a60b-4a6d-9364-b1fc171b00cd',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up datasets

Next, clean up the datasets. The code below will delete the item and interactions datasets.

In [17]:
personalize.delete_dataset(
    datasetArn = items_dataset_arn
)
personalize.delete_dataset(
    datasetArn = interactions_dataset_arn
)

{'ResponseMetadata': {'RequestId': '6e711e6a-5db4-47c4-8b89-b41408eebf81',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 03 Sep 2020 15:06:54 GMT',
   'x-amzn-requestid': '6e711e6a-5db4-47c4-8b89-b41408eebf81',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the schemas

Next, clean up the schemas. You create an interaction and item schema, delete them.

In [21]:
personalize.delete_schema(
    schemaArn = interaction_schema_arn
)
personalize.delete_schema(
    schemaArn = itemmetadataschema_arn
)

{'ResponseMetadata': {'RequestId': 'd792d2a1-2230-4996-a549-4a3b525b8e68',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 03 Sep 2020 15:09:22 GMT',
   'x-amzn-requestid': 'd792d2a1-2230-4996-a549-4a3b525b8e68',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the dataset groups

Finally, clean up the dataset group:

In [23]:
personalize.delete_dataset_group(
    datasetGroupArn = dataset_group_arn
)

{'ResponseMetadata': {'RequestId': '950aac6a-a501-4dab-90c6-06b5e78bb89d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 03 Sep 2020 15:10:46 GMT',
   'x-amzn-requestid': '950aac6a-a501-4dab-90c6-06b5e78bb89d',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the S3 bucket and IAM role

Start by deleting the role, then empty the bucket, then delete the bucket.

In [25]:
iam = boto3.client('iam')

Identify the name of the role you want to delete.

You cannot delete an IAM role which still has policies attached to it. So after you have identified the relevant role, let's list the attached policies of that role.

In [29]:
iam.list_attached_role_policies(
    RoleName = role_name
)

{'AttachedPolicies': [{'PolicyName': 'AmazonS3FullAccess',
   'PolicyArn': 'arn:aws:iam::aws:policy/AmazonS3FullAccess'}],
 'IsTruncated': False,
 'ResponseMetadata': {'RequestId': '59651918-8515-44d3-b9fc-68ea68942c30',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '59651918-8515-44d3-b9fc-68ea68942c30',
   'content-type': 'text/xml',
   'content-length': '540',
   'date': 'Thu, 03 Sep 2020 15:13:18 GMT'},
  'RetryAttempts': 0}}

You need to detach the policies in the result above using the code below. Repeat for each attached policy.

In [27]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
)

{'ResponseMetadata': {'RequestId': '7f1857d0-844f-4e14-9a2a-74aa9d7d0aa1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7f1857d0-844f-4e14-9a2a-74aa9d7d0aa1',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Thu, 03 Sep 2020 15:12:54 GMT'},
  'RetryAttempts': 0}}

In [30]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

{'ResponseMetadata': {'RequestId': 'd3d7dc1f-3f44-45fa-b32c-2e6390b440a1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd3d7dc1f-3f44-45fa-b32c-2e6390b440a1',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Thu, 03 Sep 2020 15:13:53 GMT'},
  'RetryAttempts': 0}}

Finally, you should be able to delete the IAM role.

In [31]:
iam.delete_role(
    RoleName = role_name
)

{'ResponseMetadata': {'RequestId': '7adb2824-9eea-4617-bcf3-cbe8d45e5569',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7adb2824-9eea-4617-bcf3-cbe8d45e5569',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Thu, 03 Sep 2020 15:13:57 GMT'},
  'RetryAttempts': 0}}

To delete an S3 bucket, it first needs to be empty. The easiest way to delete an S3 bucket, is just to navigate to S3 in the AWS console, delete the objects in the bucket, and then delete the S3 bucket itself.